# Baixando os dados

Para facilitar o processamento e uso das ferramentas de análise e visualização escolhi baixar tudo que é preciso primeiro. Por meio das APIs oficias da Riot Games é possível recuperar informações das partidas, assim como suas respectivas linhas do tempo. Essas informações são salvas em duas pastas em um json por partida.

APIs Utilizadas:  
SUMMONER-V4 (utilizada no próprio site)  
ACCOUNT-V1 (utilizada no próprio site)   
MATCH-V5

In [1]:
import requests
import pandas as pd
import numpy as np
import altair as alt
import json
from pathlib import Path
from datetime import datetime
import time
from tqdm.notebook import trange, tqdm

In [2]:
with open("apikey.txt", "rb") as f: 
    apikey = f.read().decode("utf-8")
with open("ODESAFIO.json","rb") as f: 
    puuid = json.loads(f.read())["puuid"]

In [3]:
rate_time = .85 # ~(50/60)

Muitas limitações foram encontradas nesse contexto usando o `datetime` do próprio python, então foi preciso trocar para o gerenciamento de tempo do `pandas`. 

In [16]:
startTime = int(pd.Timestamp('10.03.2022').timestamp())
endTime = int(pd.Timestamp('15.03.2022').timestamp())

In [17]:
startTime, endTime

(1664755200, 1647302400)

Muitos erros aconteceram utilizando o parâmetro `startTime` na URL para conseguir o ID das partidas, então para conseguir iniciar o trabalho de preprocessamento eu preferi usar somente a data final do desafio. Eventualmente isso deve ser corrigido para incluir precisamente o período.

In [7]:
matches_id = []

matches_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?endTime={endTime}&start=0&count=100&api_key={apikey}"
r = requests.get(matches_url)
matches_id = r.json()

Os laços abaixo só usam o id das partidas para recuperar a informação das partidas e linha do tempo.

In [13]:
for match_id in tqdm(matches_id):
    match_data_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={apikey}"
    r = requests.get(match_data_url)
    # time.sleep(rate_time)
    with open(f"matches/{match_id}.json", "w") as outfile:
        json.dump(r.json(), outfile, indent=4, sort_keys=True)

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
for match_id in tqdm(matches_id):
    match_data_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={apikey}"
    r = requests.get(match_data_url)
    # time.sleep(rate_time)
    with open(f"timelines/{match_id}.json", "w") as outfile:
        json.dump(r.json(), outfile, indent=4, sort_keys=True)

  0%|          | 0/100 [00:00<?, ?it/s]

Por enquanto os downloads terminam aqui, mas provavelmente um acesso dinâmico vai ser necessário na API ACCOUNT-V1 ou SUMMONER-V4 para recuperar os nomes dos outros participantes das partidas se for necessário.